In [11]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,models
import cv2
import torch.nn as nn
import torch.nn.functional as F
from scipy.stats import multivariate_normal
from scipy import random, linalg
from sklearn.model_selection import train_test_split
import torch.optim as optim
import re
import json
import time

import sys
from utils import *
#from architectures import *
from datasets import *

import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class CGData(Dataset):
    """Dataset containing all sequences with artifacts
    
    Generates three distorted images as input data.
    
    """
    
    def __init__(self, root_dir, indices, crop=False, crop_size=50, transform=None,resize=True):
        self.root_dir = root_dir
        self.transform = transform
        self.ToPIL = transforms.ToPILImage()
        self.ToTensor = transforms.ToTensor()
        self.indices = indices
        self.crop = crop
        self.crop_size = crop_size
        self.resize=resize
        
        files = os.listdir(root_dir)
        match = lambda x: len(re.findall("img_\d+_\d.jpg", x))== 1
        cut_string = lambda x: eval(re.sub("_.*","",re.sub("img_","",x)))

        files = list(filter(match,files))
        files = list(map(cut_string,files))


        first,last = min(files),max(files)
        self.offset = first
        self.last = last
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):        
        
        idx = self.indices[idx]
        count = 0
        img_files = None
        imgs = None
        label = None
        while True:
            
            n = np.random.randint(1,10)
            nrs = np.random.choice(range(1,10), size=n, replace=False).tolist()
            img_files = [self.root_dir +  "img_" +str(idx)+ "_" + str(nr) + ".jpg" for nr in nrs]
            exists = all([os.path.isfile(img_file) for img_file in img_files])
            count+=1
            try:
                imgs = [cv2.imread(file) for file in img_files]
                imgs = [img[...,::-1]- np.zeros_like(img) for img in imgs]

                label_file = self.root_dir + "books/img " + "("+str(idx - 1)+").jpg"
                label = cv2.imread(label_file)
                label = label[...,::-1]- np.zeros_like(label)
                break

            except:
                idx = np.random.randint(len(self.indices))
                idx = self.indices[idx]

        
        
        if self.resize:
            label = cv2.resize(label, dsize=(256,256))
            imgs = [ cv2.resize(img, dsize=(256,256)) for img in imgs]
        
        H,W,C = imgs[0].shape
        if H<W:
            label = np.rot90(label)
            label -= np.zeros_like(label)
            imgs = [np.rot90(img) for img in imgs]- np.zeros_like(label)
        
        flip = np.random.randint(-1,3)
        if flip < 2:
            label = cv2.flip(label,flip)- np.zeros_like(label)
            imgs = [cv2.flip(img,flip) for img in imgs]- np.zeros_like(label)

        if self.crop:
            i = np.random.randint(0,H-self.crop_size)
            j = np.random.randint(0,W-self.crop_size)
            label = label[i:i+self.crop_size,j:j+self.crop_size]
            imgs = [img[i:i+self.crop_size,j:j+self.crop_size] for img in imgs]
        
        imgs = [transforms.ToTensor()(img) for img in imgs]
        #data = torch.unsqueeze(data,0)
        
        label = label.astype(np.uint8)
        label = transforms.ToTensor()(label)
        #label = torch.unsqueeze(label,0)
        return imgs, label

# Architecture

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=True)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += x
        return out
    
class DilatedResidualBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dilation):
        super(DilatedResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1, dilation=dilation, padding=dilation, bias=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, dilation=dilation,padding=dilation, bias=True)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += x
        return out
        
class DeepSetNet(nn.Module):
    """ Deep Set Residual  Neural Network """
    def __init__(self, encoder_num_blocks=20, decoder_num_blocks=10, smooth_num_blocks=6, planes=32,block=ResidualBlock ):
        super(DeepSetNet, self).__init__()
        self.planes = planes
        self.input = nn.Conv2d(3, self.planes, kernel_size=3, stride=1, padding=1, bias=True)
        self.output= nn.Conv2d(self.planes, 3, kernel_size=3, stride=1, padding=1, bias=True)
        
        
        # Create a down-/up-sampling architecture
        self.downsample = []
        self.upsample = []
        n = planes
        for i in range(2):
            self.downsample.append( nn.Conv2d(in_channels = n, out_channels=n*2, kernel_size=3, stride=2, padding=1 ) )
            self.downsample.append(nn.ReLU(inplace=True))

            
            self.upsample = [nn.ReLU(inplace=True)] + self.upsample
            self.upsample = [nn.ConvTranspose2d(in_channels=n*2, out_channels=n, kernel_size=3, stride=2, padding=1, output_padding=1)] + self.upsample
            n *= 2

        self.downsample = nn.Sequential(*self.downsample)
        self.upsample = nn.Sequential(*self.upsample)
        
        
        # Embedding of downsampled features
        self.encoder = self._make_layer(block, n, encoder_num_blocks)
        self.decoder = self._make_layer(block, n, decoder_num_blocks)
        self.smooth  = self._make_smooth_layer(planes, smooth_num_blocks)
    def _make_layer(self, block, planes, num_blocks):
        layers = []
        for i in range(num_blocks):
            layers.append(block(planes, planes))
        return nn.Sequential(*layers)
    
    def _make_smooth_layer(self, planes, num_blocks):
        layers = []
        dilation = 0
        for i in range(num_blocks):
            if i%2 == 0:
                dilation+=1
            layers.append(DilatedResidualBlock(planes,planes,dilation))
            layers.append( nn.Conv2d(in_channels =planes, out_channels=planes, kernel_size=3, stride=1, padding=1 ) )
            layers.append(nn.ReLU(inplace=True))
            layers.append( nn.Conv2d(in_channels =planes, out_channels=planes, kernel_size=3, stride=1, padding=1 ) )
        return nn.Sequential(*layers)
            
        

    def forward(self, xs):
        """Forward pass of our DeepSet Network 
        
        xs: either a single tensor or a list of tensor of size (B, C, H, W)
        """

        if not isinstance(xs, list):
            xs = [xs]
        embedding = [self.encoder(self.downsample(self.input(x))) for x in xs]
        embedding = torch.stack(embedding).sum(0)
        out = self.output(self.smooth(self.upsample(self.decoder(embedding))))

        
        return out

In [4]:
model_dir = "./models/"
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
# while True:
#     if not os.path.isdir(model_dir):
#         os.mkdir(model_dir)
#         break
#     model_dir = "./models/"+ model_name + str(model_number)

#     model_number +=1

print("The following directory will be used in all further steps:  " + model_dir)


# Create Dataset and split it into Training and test set
minibatch_size = 5
data_dir = "E:/250x250/"
files = os.listdir(data_dir)
match = lambda x: len(re.findall("img_\d+_\d.jpg", x))== 1
cut_string = lambda x: eval(re.sub("_.*","",re.sub("img_","",x)))

files = list(filter(match,files))
files = list(map(cut_string,files))


first, last = min(files),max(files)
print(first, last)

n = last - first + 1
train, test = train_test_split(range(first, last+1))
#train, test = train_test_split(range(first, first+200))

if os.path.isfile(model_dir + "/trainingIdx.txt"):
    f1 = open(model_dir + "/trainingIdx.txt", "r")
    f2 = open(model_dir + "/testIdx.txt", "r")
    train = eval(f1.read())
    test = eval(f2.read())
    f1.close()
    f2.close()
else:
    write(model_dir + "/trainingIdx.txt",train)
    write(model_dir + "/testIdx.txt",test)

    
if os.path.isfile(model_dir + "/params.json"):
    f = open(model_dir + "/params.json", "r")
    params = json.loads(f.read())
    f.close()
else:
    params = {
        "epoch": 0,
        "time": 0,
        "best_loss": np.inf
    }
    f = open(model_dir + "/params.json", "w")
    f.write(json.dumps(params))
    f.close()

train_set = CGData(data_dir,train)
test_set = CGData(data_dir,test)


device = torch.device("cuda:0")
 
# Initialize Neural Network
net = DeepSetNet(encoder_num_blocks=10, decoder_num_blocks=5, planes=64)

if  os.path.isfile(model_dir + "/nn.pt"):
    net.load_state_dict(torch.load(model_dir + "/nn.pt"))

net = net.to(device)

criterion = nn.MSELoss()
epochs = 10000

The following directory will be used in all further steps:  ./models/
6462 60848


In [5]:
net

DeepSetNet(
  (input): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (output): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (downsample): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (upsample): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3

In [ ]:
optimizer = optim.Adam(net.parameters(),lr = 0.0001,)
optimizer.zero_grad()

def training(model,train_set,test_set,
             model_dir,optimizer,
             params,
             criterion= nn.MSELoss(),
             epochs=1000,
             device=torch.device("cuda:0")):
    
    last_epoch = params["epoch"]
    best_loss =  params["best_loss"]
    t = params["time"]

    
    train_loss = 0
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.1)
    
    # This simulates the Learning rate updates
    for _ in range(last_epoch):
        scheduler.step()
    

    for epoch in range(last_epoch+1,epochs):
        
        start = time.time()
        samples = 0
        train_loss = 0
        for i,(x,y) in enumerate(train_set):
            
            x = [img.unsqueeze(0).to(device) for img in x]
            y = y.unsqueeze(0).to(device)
            output = net.forward(x)
            loss = criterion(output,y)
            train_loss += loss.item()
            loss.backward()
            
            if (i+1)%50 == 0 or i == len(train_set)-1:
                optimizer.step()
                optimizer.zero_grad()
            samples +=1
            update_progress(i/len(train_set))
            
            
        train_loss = train_loss/samples
        samples = 0
        net.eval()
        test_loss = 0
        with torch.no_grad():
               for x,y in test_set:
                #x,y = train_set[i]
                x = [img.unsqueeze(0).to(device) for img in x]
                y = y.unsqueeze(0).to(device)
                output = net.forward(x)
                loss = criterion(output,y)
                test_loss += loss.item()
                samples +=1
        test_loss = test_loss/samples
        
        end = time.time()
        t += (end - start)
        print("Epoch:%d Train-Error: %f , Test-Error:%f , Time: %f" % (epoch,train_loss,test_loss,t))
        write(model_dir + "/errors.csv",[epoch,train_loss,test_loss,t])
        


        
        #torch.save(net.state_dict(), model_dir + "/nn"+ str(epoch) +".pt")
        if(test_loss<best_loss):
            torch.save(net.state_dict(), model_dir + "/nn.pt")
            best_loss = test_loss
            
        params["best_loss"] = best_loss
        params["epoch"] = epoch
        params["time"] = t
        f = open(model_dir + "/params.json", "w")
        f.write(json.dumps(params))
        f.close()
        #if(train_loss>2*best_loss):
            #net.load_state_dict(torch.load(model_dir + "/nn.pt"))

        net.train()
        scheduler.step()


training(net, train_set,test_set,model_dir,optimizer,params,criterion)

Progress: [#########-----------] 43.7%
